In [34]:
import requests
import json
import pprint

API_KEY = "9eacbe0f-7c79-421b-a1cb-303d3d2d79fd"

url = "https://api.harvardartmuseums.org/classification"

params = {
    "apikey":API_KEY,
    "size":100
}

response = requests.get(url, params)
class_data = response.json()

for i in class_data["records"]:
  if i["objectcount"] > 2500:
    print(i)

{'objectcount': 2859, 'name': 'Accessories (non-art)', 'id': 2, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 2}
{'objectcount': 85773, 'name': 'Photographs', 'id': 17, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 17}
{'objectcount': 33969, 'name': 'Drawings', 'id': 21, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 21}
{'objectcount': 72238, 'name': 'Prints', 'id': 23, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 23}
{'objectcount': 6887, 'name': 'Paintings', 'id': 26, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 26}
{'objectcount': 6643, 'name': 'Sculpture', 'id': 30, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 30}
{'objectcount': 20270, 'name': 'Coins', 'id': 50, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 50}
{'objectcount': 6223, 'name': 'Vessels', 'id': 57, 'lastupdate': '2025-11-24T04:03:13-05:00', 'classificationid': 57}
{'objectcount': 3445, 'name': 'Tex

In [35]:
API_KEY = "9eacbe0f-7c79-421b-a1cb-303d3d2d79fd"

link = "https://api.harvardartmuseums.org/object"


In [36]:
Fragment_records = []

for i in range(1,26):
  params1= {
      "apikey":API_KEY,
      "size":100,
      "page":i,
      "division":"Fragments"
  }

  Frag = requests.get(link,params= params1)
  Fragment_data = Frag.json()
  if "records" in Fragment_data:
    Fragment_records.extend(Fragment_data["records"])

In [37]:
Fragment_md = []
Fragment_media = []
Fragment_colors = []

for i in Fragment_records:
    Fragment_md.append(dict(
         object_id = i.get('id'),
         title = i.get('title'),
         culture = i.get('culture'),
         period =i.get('period'),
         century =i.get('century'),
         medium = i.get('medium'),
         dimension = i.get('height','width'),
         description = i.get('description'),
         department = i.get('department'),
         classification = i.get('classification'),
         accessionyear = i.get('accessionyear'),
         accessionmethod = i.get('accessionmethod')
    ))
    Fragment_media.append(dict(
        object_id = i.get('objectid'),
        imagecount = i.get('imagecount'),
        metacount = i.get('metacount'),
        colourcount = i.get('colourcount'),
        rank = i.get('rank'),
        datebegin = i.get('datebegin'),
        dateend = i.get('dateend')
    ))
    color_details = i.get('colors')
    if color_details:
        for j in color_details:
            Fragment_colors.append(dict(
                object_id = i.get('objectid'),
                colour = j.get('colour'),
                spectrum = j.get('spectrum'),
                hue = j.get('hue'),
                percent = j.get('percent'),
                css3 = j.get('css3')
            ))

In [29]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 1.6 MB/s eta 0:00:00m eta 0:00:010:00:01


In [38]:
import mysql.connector
mysql = mysql.connector.connect(
    host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
    user = "okrkj9Aqr6VZPb8.root",
    password = "UkgRlt2NTpXZY999",
    port = 4000
)

In [39]:
sql = mysql.cursor()
sql.execute("CREATE DATABASE IF NOT EXISTS Harward_Project")
sql.execute("USE Harward_Project")

In [41]:
# Drop tables in reverse dependency order
sql.execute("DROP TABLE IF EXISTS Fragment_colors")
sql.execute("DROP TABLE IF EXISTS Fragment_media")
sql.execute("DROP TABLE IF EXISTS Fragment_md")

In [42]:
# Recreate Fragment_md_table with increased title column size
sql.execute("""
CREATE TABLE Fragment_md (
    object_id INT PRIMARY KEY,
    title TEXT,
    culture VARCHAR(255),
    period VARCHAR(255),
    century VARCHAR(255),
    medium TEXT,
    dimension TEXT,
    description TEXT,
    department VARCHAR(255),
    classification VARCHAR(255),
    accessionyear INT,
    accessionmethod VARCHAR(255)
)
""")

# Recreate Fragment_media_table
sql.execute("""
CREATE TABLE Fragment_media (
    object_id INT PRIMARY KEY,
    imagecount INT,
    metacount INT,
    colourcount INT,
    object_rank INT,
    datebegin INT,
    dateend INT,
    FOREIGN KEY (object_id) REFERENCES Fragment_md(object_id)
)
""")

# Recreate Fragement_colors_table
sql.execute("""
CREATE TABLE Fragment_colors (
    id INT AUTO_INCREMENT PRIMARY KEY,
    object_id INT,
    colour VARCHAR(255),
    spectrum VARCHAR(255),
    hue VARCHAR(255),
    percent FLOAT,
    css3 VARCHAR(255),
    FOREIGN KEY (object_id) REFERENCES Fragment_md(object_id)
)
""")

In [43]:
insert_query_md = """
INSERT INTO Fragment_md (
    object_id, title, culture, period, century, medium, dimension, description, department, classification, accessionyear, accessionmethod
)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    title = VALUES(title),
    culture = VALUES(culture),
    period = VALUES(period),
    century = VALUES(century),
    medium = VALUES(medium),
    dimension = VALUES(dimension),
    description = VALUES(description),
    department = VALUES(department),
    classification = VALUES(classification),
    accessionyear = VALUES(accessionyear),
    accessionmethod = VALUES(accessionmethod)
"""

# Prepare data for insertion
fragment_md_data = []
for record in Fragment_md:
    fragment_md_data.append((
        record.get('object_id'),
        record.get('title'),
        record.get('culture'),
        record.get('period'),
        record.get('century'),
        record.get('medium'),
        record.get('dimension'),
        record.get('description'),
        record.get('department'),
        record.get('classification'),
        record.get('accessionyear'),
        record.get('accessionmethod')
    ))

# Execute the insertion
sql.executemany(insert_query_md, fragment_md_data)

In [44]:
insert_query_media = """
INSERT INTO Fragment_media (
    object_id, imagecount, metacount, colourcount, object_rank, datebegin, dateend
)
VALUES (%s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE
    imagecount = VALUES(imagecount),
    metacount = VALUES(metacount),
    colourcount = VALUES(colourcount),
    object_rank = VALUES(object_rank),
    datebegin = VALUES(datebegin),
    dateend = VALUES(dateend)
"""

# Prepare data for insertion
fragment_media_data = []
for record in Fragment_media:
    fragment_media_data.append((
        record.get('object_id'),
        record.get('imagecount'),
        record.get('metacount'),
        record.get('colourcount'),
        record.get('rank'), # Using 'rank' from the original data which maps to 'object_rank'
        record.get('datebegin'),
        record.get('dateend')
    ))

# Execute the insertion
sql.executemany(insert_query_media, fragment_media_data)

In [45]:
insert_query_colors = """
INSERT INTO Fragment_colors (
    object_id, colour, spectrum, hue, percent, css3
)
VALUES (%s, %s, %s, %s, %s, %s)
"""

# Prepare data for insertion
fragment_colors_data = []
for record in Fragment_colors:
    fragment_colors_data.append((
        record.get('object_id'),
        record.get('colour'),
        record.get('spectrum'),
        record.get('hue'),
        record.get('percent'),
        record.get('css3')
    ))

# Execute the insertion
sql.executemany(insert_query_colors, fragment_colors_data)
mysql.commit() # Commit the changes to the database

# **OTHER OBJECTS**

In [4]:
Manuscript_records = []

for i in range(1,26):
  params2= {
      "apikey":API_KEY,
      "size":100,
      "page":i,
      "division":"Fragments"
  }

  Manu = requests.get(link,params= params2)
  Manuscript_data = Manu.json()
  if "records" in Manuscript_data:
    Manuscript_records.extend(Manuscript_data["records"])

In [5]:
Photo_records = []

for i in range(1,26):
  params3= {
      "apikey":API_KEY,
      "size":100,
      "page":i,
      "division":"Photograph"
  }

  Photo = requests.get(link,params= params3)
  Photograph_data = Photo.json()
  if "records" in Photograph_data:
    Photo_records.extend(Photograph_data["records"])

In [6]:
Print_records = []

for i in range(1,26):
  params4= {
      "apikey":API_KEY,
      "size":100,
      "page":i,
      "division":"Prints"
  }

  Print = requests.get(link,params= params4)
  Print_data = Print.json()
  if "records" in Print_data:
    Print_records.extend(Print_data["records"])

In [7]:
Coins_records = []

for i in range(1,26):
  params5= {
      "apikey":API_KEY,
      "size":100,
      "page":i,
      "division":"Coins"
  }

  Coins = requests.get(link,params= params5)
  Coins_data = Coins.json()
  if "records" in Coins_data:
    Coins_records.extend(Coins_data["records"])